In [ ]:
import spot
spot.setup()
import buddy
from spot.jupyter import display_inline

from decimal import Decimal
import decimal
from math import inf

from fimdp.consMDP import ConsMDP
from fimdp.labeledConsMDP import LCMDP

# Product of lCMDP and DBA with the link to LTL

The goal of this notebook is, given an LTL formula over the set $AP$ of atomic proposition and a consumption MDP with states labeled by subsets of $AP$, decide if there is a strategy for the MDP such that the LTL formula is satisfied with probability 1. We ilustrate the whole concept of a running example in which we want to enforce visiting 2 states infinitely often.

Let's first create a CMDP, we will use the following function for easier definitions of actions using uniform distributions.

In [ ]:
def uniform(dests):
    """Create a uniform distribution for given destinations.
    
    dests: iterable of states
    """
    count = len(dests)
    mod = 100 % count
    decimal.getcontext().prec = 2
    prob = Decimal(1)/Decimal(count)
    dist = {i: prob for i in dests}
    last = dests[-1]
    dist[last] = dist[last] + Decimal("0.01")*mod
    return dist

In the following code, we verify that we can achieve the Büchi objective with targets set `{1,2}` with capacity `5` and that is not enough to visit the state `1`. What we actualy want is to visit **both** of these states infinitely often which we solve later.

In [ ]:
mdp = ConsMDP()
mdp.new_states(4)
mdp.set_reload(3)
mdp.add_action(0, uniform([1,2]), "α", 3)
mdp.add_action(0, uniform([2,3]), "β", 1)
mdp.add_action(1, uniform([3]), "r", 3)
mdp.add_action(2, uniform([3]), "r", 1)
mdp.add_action(3, uniform([0]), "s", 3)
mdp.get_Buchi([1,2], 5, True)
mdp

The corresponding strategy confirms that the state 1 won't be visited by the strategy as there is no occurence of the action `α`.

In [ ]:
mdp.energy_levels.get_strategy(4, True)

## LTL and Büchi automata
Our goal of visiting both states `1` \& `2` infinitely often can be expressed by the LTL formula $\mathsf{G}\mathsf{F} s_1 \land \mathsf{G}\mathsf{F}s_2$ (or in the box-diamond notation: $\Box \diamond s_1 \land \Box \diamond s_2$) where the atomic proposition $s_1$ corresponds to visiting state `1` and the tomic proposition $s_2$ corresponds to visiting state`2`.

This formula can be expressed by a **deterministic** üchi automaton (DBA). We use Spot to make the translation for us. The option `BA` forces Spot to deliver a state-based Büchi automaton (default is transition-based generalized Büchi automaton), the option `deterministic` indicates that we prefer deterministic automata, and `complete` asks for an automaton with complete transition function. If you are not sure that your formula can be translated to a DBA, consult [hierarchy of LTL](https://spot.lrde.epita.fr/hierarchy.html). It is also a good practice to make yourself sure by running 
```python
aut.is_deterministic()
```

In [ ]:
f = spot.formula("GF s1 & GF s2")
aut = spot.translate(f, "BA", "deterministic", "complete")

In [ ]:
display(aut, aut.is_deterministic())

The produced automaton can be used in parallel with our input MDP; this is achieved by a _product_ (alternatively _parallel synchonous composition_) of this automaton an the input MDP. But we need to label states of the MDP with the atomic propositions `s₁` and `s₂`.

## Labeled CMDP
We create a copy of our CMDP and label the states `1` and `2` with the corresponding atomic propositions using the function
```python
LCMDP.state_labels(labels)
```
where `labels` is a list (of length equal to number of states) of sets of ints; the ints are indices to the list `AP` given in the constructor of `LCMDP`.

In [ ]:
lmdp = LCMDP(AP=["s1","s2"], mdp=mdp)
lmdp.state_labels = [set(), {0}, {1}, set()]
display(lmdp, lmdp.state_labels)

## Product of labeled CMDP and DBA

In the following, we explain and show the (simplified) implementation of `LCMDP.product(self, dba)`.

The states of the product are tuples `(ms,as)` where `ms` stands for a state of the MDP and `as` stands for a states of the automaton. Let's call the set of states of the MDP $S$ and the set of states of the DBA $Q$; further, the labeling function of the labeled MDP is $\lambda \colon S \to 2^{AP}$ and the transition function of the DBA as $\delta \colon Q \times 2^{AP} \to Q$. For each action `α` and each successor `ms'` for this action from state `ms`, the action `α` of `(ms,as)` has an `α` successor (with the same probability) `(ms', as')` where `as'` is equal to $\delta(as, \lambda(ms'))$.

All tuples that contain a reload state of the mdp, are again reloading. All tuples with an accepting state of the automaton will become targets. The following function `product(lmdp, aut)` returns a CMDP that is the product of `lmdp` and `aut` and a list of target states.

We can now see the result of the product on the labeled MDP and the automaton for $\mathsf{G}\mathsf{F}s_1 \land \mathsf{G}\mathsf{F} s_2$. We can also see that capacity 5 is no longer sufficient for the Büchi objective (the green ∞ indicate that no initial load is sufficient from given state to satisfy the Büchi objectives with targets `T`). In fact, we need at least 9 units of energy to pass the path through mdp-state `1`.

In [ ]:
p, T = lmdp.product(aut)
res = p.get_Buchi(T, 5, True)
display_inline(p)

In [ ]:
assert res == [inf, inf, inf, inf, inf, inf, inf]

In [ ]:
p, T = lmdp.product(aut)
res = p.get_Buchi(T, 9, True)
display_inline(p)

In [ ]:
assert p.names == ['0,1', '1,0', '2,1', '3,1', '3,0', '0,0', '2,2']
assert res == [2, 3, 1, 0, 0, 2, 1]

## Test strategy

In [ ]:
from fimdp.energy_solver import BUCHI
res = p.energy_levels.get_strategy(BUCHI)
assert res == [{6: 'α', 2: 'β'}, {3: 'r'}, {1: 'r'}, {0: 's'}, {0: 's'}, {2: 'β'}, {1: 'r'}]